In [1]:
from art.attacks.extraction import CopycatCNN
from art.estimators.classification import KerasClassifier
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.models import load_model, Sequential
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

/Users/oscarsaravia/Documents/Universidad/security-data-science/SDS-LAB07/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/oscarsaravia/Documents/Universidad/security-data-science/SDS-LAB07/venv/lib/python3.11/site-packages/art/estimators/certification/__init__.py:14: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


In [2]:
# Carga del modelo creado
use_model = load_model('./model')
load_x = np.load('x_val.npy')
load_y = np.loadtxt('y_val.txt', delimiter=',')
X_train, X_test, y_train, y_test_ = train_test_split(load_x, load_y, test_size=0.30, random_state=1)

2023-05-17 22:28:30.097686: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-05-17 22:28:30.107744: W tensorflow/c/c_api.cc:304] Operation '{name:'AssignVariableOp_2' id:107 op device:{requested: '/device:CPU:0', assigned: ''} def:{{{node AssignVariableOp_2}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false, _device="/device:CPU:0"](count_1, Identity_2)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [3]:
# Creacion del clasificador
keras_classifier = KerasClassifier(use_model)
copyCat = CopycatCNN(batch_size_fit=16, batch_size_query=16, nb_epochs=5,nb_stolen=500,classifier=keras_classifier)

In [4]:
sequential_model = Sequential()
sequential_model.add(Convolution2D(32, (3, 3), strides=(1, 1), input_shape=(256, 256, 3), activation='relu'))
sequential_model.add(MaxPooling2D(pool_size=(2, 2)))
sequential_model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
sequential_model.add(MaxPooling2D(pool_size=(2, 2)))
sequential_model.add(Flatten())
sequential_model.add(Dense(512, activation='softplus'))
sequential_model.add(Dense(25, activation='sigmoid'))
sequential_model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [5]:
model_stolen = KerasClassifier(model=sequential_model)

In [6]:
stolen_classifier = copyCat.extract(
  x=X_train, 
  y=y_train, 
  thieved_classifier=model_stolen
)

/Users/oscarsaravia/Documents/Universidad/security-data-science/SDS-LAB07/venv/lib/python3.11/site-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-05-17 22:28:37.073640: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_1/Sigmoid' id:150 op device:{requested: '', assigned: ''} def:{{{node dense_1/Sigmoid}} = Sigmoid[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_1/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-17 22:28:37.140206: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_2/bias/Assign' id:386 op device:{requested: '', assigned: ''} def:{{{node dense_2/bias/Assign}} = AssignVariableOp[_has_man

Train on 500 samples


2023-05-17 22:28:42.691756: W tensorflow/c/c_api.cc:304] Operation '{name:'loss_1/mul' id:492 op device:{requested: '', assigned: ''} def:{{{node loss_1/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_1/mul/x, loss_1/dense_1_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-17 22:28:42.712823: W tensorflow/c/c_api.cc:304] Operation '{name:'training/Adam/learning_rate/Assign' id:669 op device:{requested: '', assigned: ''} def:{{{node training/Adam/learning_rate/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/learning_rate, training/Adam/learning_rate/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. 

Epoch 1/5
500/500 [==============================] - 18s 37ms/sample - loss: 263.7064 - accuracy: 0.9460
Epoch 2/5
500/500 [==============================] - 18s 36ms/sample - loss: 119.6243 - accuracy: 0.9940
Epoch 3/5
500/500 [==============================] - 17s 35ms/sample - loss: 3.0368 - accuracy: 0.9980
Epoch 4/5
500/500 [==============================] - 17s 34ms/sample - loss: 0.5388 - accuracy: 0.9820
Epoch 5/5
500/500 [==============================] - 16s 32ms/sample - loss: 0.2154 - accuracy: 0.9980


In [8]:
score1 = keras_classifier._model.evaluate(x=X_test, y=y_test_)
score2 = stolen_classifier._model.evaluate(x=X_test,y=y_test_)
print(score1)
print(score2)